In [1]:
import duckdb
import pandas as pd

In [3]:
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql 

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Cargar el dataset:

In [5]:
dataset_path = 'C:/Eugenio/Maestria/DMEyF/datasets/'
dataset_file = 'competencia_01_crudo.csv'

In [6]:
%%sql

create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


Verificar que todo este bien:

In [7]:
%sql select * from competencia_01_crudo limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,202101,1,0,0,61,300,1664.94,17994.40,1605.81,...,0.0,4,0.0,-37098.90,0.0,7867,118514.88,10,0,76538.25
1,249221468,202101,1,0,0,53,37,4879.78,18640.95,1149.08,...,0.0,4,0.0,-9864.93,0.0,1124,31427.14,14,0,0.00
2,249223005,202101,1,0,0,48,202,1828.35,37789.01,610.12,...,0.0,4,0.0,-32119.08,0.0,3486,855.11,2,0,19389.69
3,249228180,202101,1,0,0,66,320,175.87,2600.67,276.22,...,0.0,4,0.0,-10891.30,0.0,8925,7517.73,6,0,516.12
4,249232117,202101,1,0,0,79,373,16600.69,32270.07,16367.13,...,0.0,4,0.0,-43610.75,0.0,8607,37309.25,5,0,3084.99


In [9]:
%%sql
select
    foto_mes
    , count(*) as cantidad # cuenta cuantos casos hay en cada foto_mes
                           # y lo guarda en un campo llamado cantidad
from competencia_01_crudo
group by foto_mes

,foto_mes,cantidad
0,202101,162026
1,202102,162646
2,202103,163685
3,202104,164090
4,202105,164623
5,202106,164876


Crear la clase ternaria:

In [10]:
%%sql

create or replace table competencia_01 as

with periodos as (
    select distinct foto_mes from competencia_01_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0 # genero la columna mes_0 y sobre ella genero mes_1 y mes_2
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_2 = 1, 'CONTINUA',
             if(mes_1 = 0, 'BAJA+1',
                if(mes_2 = 0,'BAJA+2',NULL))) as clase_ternaria # Replazar null por la lógica que genera el target
    from todo t
    left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
) 

select * EXCLUDE (mes_0, mes_1, mes_2) # descarto las columnas auxiliares
from clase_ternaria
where mes_0 = 1 # descarto las filas de los clientes para los meses previos a que ingresaran al banco.

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Verificamos:

In [13]:
%sql select count(*) from competencia_01

,count_star()
0,981946


In [11]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,635,785,160606
1,202102,785,1017,160844
2,202103,1020,981,161684
3,202104,982,1189,161919
4,202105,1192,0,0
5,202106,0,0,0


Guardar el dataset modificado:

In [15]:
%sql COPY competencia_01 TO '{{dataset_path}}competencia_01.csv' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
